In [19]:
!pip install pyECLAT upsetplot
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt
from pyECLAT import ECLAT

In [20]:
df = pd.read_csv('/content/drive/MyDrive/Groceries_dataset.csv')

print(df.head())
print(df.info())

print(f"Unique Items: {df['itemDescription'].nunique()}")
print(f"Unique Members: {df['Member_number'].nunique()}")
print(f"Unique Dates: {df['Date'].nunique()}")

   Member_number        Date   itemDescription
0           1808  21-07-2015    tropical fruit
1           2552  05-01-2015        whole milk
2           2300  19-09-2015         pip fruit
3           1187  12-12-2015  other vegetables
4           3037  01-02-2015        whole milk
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38765 entries, 0 to 38764
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Member_number    38765 non-null  int64 
 1   Date             38765 non-null  object
 2   itemDescription  38765 non-null  object
dtypes: int64(1), object(2)
memory usage: 908.7+ KB
None
Unique Items: 167
Unique Members: 3898
Unique Dates: 728


In [21]:
from collections import defaultdict

# Step 1: Group transactions by Member + Date
transactions = df.groupby(
    ['Member_number', 'Date']
)['itemDescription'].apply(list).reset_index()

# Step 2: Build vertical data = item → set of transaction indices
vertical_data = defaultdict(set)

for idx, row in transactions.iterrows():
    items = set(row['itemDescription'])  # ensure unique items per transaction
    for item in items:
        vertical_data[item].add(idx)

# Print vertical data example
for item, tids in vertical_data.items():
    print(f"{item} → {tids}")

# vertical_data now looks like:
# milk → {0, 3, 7}, bread → {0, 2, 4}, ...
# where the set values are indices of the transactions list

sausage → {0, 2048, 12290, 3, 12291, 6, 4102, 12295, 10250, 14, 12305, 12307, 6164, 4117, 6166, 10263, 8218, 2077, 6177, 14372, 12331, 6188, 10284, 4147, 6196, 6201, 58, 10298, 12345, 4157, 6205, 14396, 8256, 10307, 14406, 2120, 2126, 6226, 14420, 85, 12379, 93, 10333, 2144, 98, 6244, 6245, 12388, 4199, 8295, 4203, 10348, 2157, 6253, 14446, 14447, 8306, 2163, 4214, 6264, 8312, 12409, 124, 4220, 131, 12419, 12421, 2182, 6279, 2184, 6280, 141, 2190, 6288, 10386, 12436, 4249, 8345, 2204, 2206, 14500, 6310, 14503, 10410, 14509, 6318, 4271, 12463, 14510, 2231, 14522, 2236, 12477, 12482, 10436, 8390, 12489, 2250, 4306, 6358, 215, 10454, 14552, 14554, 220, 8419, 12520, 10481, 8434, 6394, 2299, 4347, 8444, 10490, 12541, 12543, 6402, 6406, 8458, 4366, 8462, 272, 14609, 8466, 275, 12562, 8471, 8472, 14621, 8486, 4393, 12586, 10539, 14635, 8493, 14638, 311, 6455, 8505, 10553, 315, 12602, 322, 12610, 2373, 4421, 327, 4423, 329, 330, 14667, 8526, 12627, 8532, 344, 12280, 14685, 8543, 8549, 10599, 1

In [22]:
import pandas as pd
horizontal_transactions = transactions['itemDescription'].tolist()
eclat_df = pd.DataFrame(horizontal_transactions)

In [ ]:
from pyECLAT import ECLAT

eclat = ECLAT(data=eclat_df, verbose=True)
min_support = 0.03

freq_itemsets, support_dict = eclat.fit(
    min_support=min_support,
    min_combination=1,
    max_combination=2
)

print("Frequent Itemsets Found:", freq_itemsets)

 83%|████████▎ | 139/168 [00:01<00:00, 129.98it/s]

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Build a DataFrame from itemsets & their support
support_df = pd.DataFrame([
    {"itemset": ", ".join(itemset), "support": support_dict[itemset]}
    for itemset in freq_itemsets
])

# Select the first 10 itemsets (order as returned by pyECLAT)
first10_df = support_df.head(10).reset_index(drop=True)

# Plot the first 10 itemsets
plt.figure(figsize=(12, 8))
plt.barh(first10_df["itemset"], first10_df["support"], color="skyblue")
plt.title("First 10 Frequent Itemsets (in ECLAT Output Order)")
plt.xlabel("Support (Count)")
plt.ylabel("Itemset")
plt.gca().invert_yaxis()  # largest at top
plt.show()

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Take only the first 10 items from vertical_data
top_items = list(vertical_data.keys())[:10]
n = len(top_items)

# Build matrix of intersection sizes for just those 10
matrix = np.zeros((n, n), dtype=int)

for i in range(n):
    for j in range(n):
        if i <= j:  # fill symmetrically
            intersection = vertical_data[top_items[i]].intersection(vertical_data[top_items[j]])
            matrix[i, j] = matrix[j, i] = len(intersection)

# Plot heatmap
plt.figure(figsize=(10, 10))  # square shape works better for 10×10
sns.heatmap(
    matrix,
    annot=True,
    fmt="d",
    xticklabels=top_items,
    yticklabels=top_items,
    cmap="Blues"
)
plt.title("Pairwise Tidset Intersections (Top 10 Items)")
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import matplotlib.pyplot as plt
from upsetplot import from_contents, plot

# 1) Get the top 10 items by support (largest tidsets)
top_5_items = sorted(
    vertical_data.keys(),
    key=lambda i: len(vertical_data[i]),
    reverse=True
)[:10]

# 2) Build contents only for these top items
contents_top10 = {item: list(vertical_data[item]) for item in top_10_items}

# 3) Create UpSet‑plot data
upset_data_top10 = from_contents(contents_top10)

# 4) Plot
plot(upset_data_top10)
plt.title("UpSet Plot — Top 5 Item Tidset Intersections")
plt.show()

In [ ]:
from functools import partial

# global counter to stop after printing 10 itemsets
counter = {"count": 0}

def print_eclat_tree_limited(prefix, items, vertical_data, min_support, limit, counter):
    # Stop recursion if limit reached
    if counter["count"] >= limit:
        return

    for i, item in enumerate(items):
        # Stop early if we’ve printed enough
        if counter["count"] >= limit:
            return

        new_prefix = prefix + [item]

        # determine tidset for this new itemset
        if len(prefix) == 0:
            current_tidset = vertical_data[item]
        else:
            prefix_tidset = set.intersection(*(vertical_data[x] for x in prefix))
            current_tidset = prefix_tidset & vertical_data[item]

        # check support
        if len(current_tidset) >= min_support:
            print(f"{new_prefix} (support={len(current_tidset)})")
            counter["count"] += 1

            # if we still haven’t hit the limit, recurse to expand itemsets
            if counter["count"] < limit:
                print_eclat_tree_limited(
                    new_prefix,
                    items[i+1:],
                    vertical_data,
                    min_support,
                    limit,
                    counter
                )

# run the limited tree
print("First 10 Frequent Itemsets in ECLAT DFS Order (min_support={}):".format(min_support))
print_eclat_tree_limited([], list(vertical_data.keys()), vertical_data, min_support, 10, counter)